# Operar con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas elemento a elemento, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.).
**Pandas hereda gran parte de esta funcionalidad de NumPy, y las ufuncs** que introdujimos en [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) son clave para ello.

Pandas incluye un par de giros útiles, sin embargo: para operaciones unarias como negación y funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para operaciones binarias como **adición y multiplicación, Pandas *alineará automáticamente los índices* al pasar los objetos a la ufunc.**
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes -ambas tareas potencialmente propensas a errores con arrays NumPy sin procesar- se convierten en tareas esencialmente infalibles con Pandas.
Además, veremos que hay operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Ufuncs: Índice Preservación

Como Pandas está diseñado para trabajar con NumPy, cualquier ufunc de NumPy funcionará con los objetos ``Series`` y ``DataFrame`` de Pandas.
Empecemos definiendo una ``Series`` y un ``DataFrame`` simples para demostrarlo:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

In [ ]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

Si aplicamos una ufunc NumPy sobre cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*

In [ ]:
np.exp(ser)

O, para un cálculo un poco más complejo:

In [ ]:
np.sin(df * np.pi / 4)

Cualquiera de las ufuncs discutidas en [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) puede ser usada de forma similar.

## UFuncs: Index Alignment

Para operaciones binarias sobre dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realización de la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos que siguen.

### Alineación del índice en la serie

Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes, y encontramos sólo los tres primeros estados de EE.UU. por *área* y los tres primeros estados de EE.UU. por *población*:

In [ ]:
area = pd.Series({'Alaska': 1723337,
                  'Texas': 695662,
                  'California': 423967},
                 name='area')

population = pd.Series({'California': 38332521,
                        'Texas': 26448193,
                        'New York': 19651127},
                       name='population')

Veamos qué ocurre cuando los dividimos para calcular la densidad de población:

In [ ]:
type(population / area)

In [ ]:
s_poblacion_area = population / area
print(s_poblacion_area)

La matriz resultante contiene la *unión* de los índices de las dos matrices de entrada, que podría determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:

In [ ]:
area.index

In [ ]:
population.index

In [ ]:
area.index | population.index

In [ ]:
area.index.union(population.index)

Cualquier elemento para el que uno u otro no tenga una entrada se marca con ``NaN``, o "Not a Number", que es como Pandas marca los datos que faltan (ver más discusión sobre datos que faltan en [Handling Missing Data](03.04-Missing-Values.ipynb)).
Esta coincidencia de índices se implementa de esta forma para cualquiera de las expresiones aritméticas incorporadas en Python; cualquier valor que falte se rellena con NaN por defecto:

In [ ]:
A = pd.Series([2, 4, 6], index=["andalucia", "aragon", "madrid"])
print(A)
B = pd.Series([1, 3, 5], index=["aragon", "madrid", "asturias"])
print(B)
A + B

In [ ]:
df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB

In [ ]:
# df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB['n_personas'] = df_AB['n_alumnos'] + df_AB['n_profesores']
df_AB

In [ ]:
df_AB['n_alumnos_10'] = df_AB['n_alumnos'] * 10
df_AB

Si el uso de valores NaN no es el comportamiento deseado, el valor de relleno puede modificarse utilizando métodos de objeto apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento de ``A`` o ``B`` que pueda faltar:

In [ ]:
B + A

In [ ]:
print(A)
print(B)

In [ ]:
A.add(B, fill_value=0)

### Alineación de índices en DataFrame

Un tipo similar de alineación tiene lugar para *ambos* columnas e índices cuando se realizan operaciones en ``DataFrame``s:

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

In [ ]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

In [ ]:
A + B

Observe que los índices se alinean correctamente independientemente de su orden en los dos objetos, y que los índices del resultado están ordenados.
Como en el caso de ``Series``, podemos utilizar el método aritmético del objeto asociado y pasarle cualquier ``valor_de_relleno`` que queramos utilizar en lugar de las entradas que falten.
Aquí rellenaremos con la media de todos los valores de ``A`` (calculada apilando primero las filas de ``A``):

In [ ]:
fill = A.values.mean()
A.add(B, fill_value=fill)

La siguiente tabla lista los operadores de Python y sus métodos equivalentes en los objetos de Pandas:

| Python Operador | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Cuando se realizan operaciones entre un ``DataFrame`` y una ``Series``, la alineación de índices y columnas se mantiene de forma similar.
Las operaciones entre un ``DataFrame`` y una ``Series`` son similares a las operaciones entre un array NumPy bidimensional y unidimensional.
Consideremos una operación común, donde encontramos la diferencia de un array bidimensional y una de sus filas:

In [ ]:
A = rng.randint(10, size=(3, 4))
A

In [ ]:
A - A[0]

Según las reglas de difusión de NumPy (véase [Computación en matrices: difusión](02.05-Computation-on-arrays-broadcasting.ipynb)), la resta entre una matriz bidimensional y una de sus filas se aplica fila a fila.

En Pandas, la convención opera de forma similar por defecto:

In [ ]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

Si, por el contrario, desea operar por columnas, puede utilizar los métodos de objeto mencionados anteriormente, especificando la palabra clave ``axis``:

In [ ]:
df.subtract(df['R'], axis=0)

Tenga en cuenta que estas operaciones ``DataFrame``/``Series``, al igual que las operaciones comentadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [ ]:
df

In [ ]:
halfrow = df.iloc[0, ::2]
halfrow

In [ ]:
df - halfrow

Esta preservación y alineación de índices y columnas significa que las operaciones sobre datos en Pandas siempre mantendrán el contexto de los datos, lo que previene los tipos de errores tontos que podrían surgir al trabajar con datos heterogéneos y/o desalineados en arrays NumPy sin procesar.